In [2]:
!pip install --upgrade --pre pythainlp
!pip install pyLDAvis

     --------------------------------------- 13.8/13.8 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pythainlp
    Found existing installation: pythainlp 4.1.0b3
    Uninstalling pythainlp-4.1.0b3:
      Successfully uninstalled pythainlp-4.1.0b3
  Using cached numpy-1.25.2-cp310-cp310-win_amd64.whl (15.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mindy\\anaconda3\\Lib\\site-packages\\~-mpy\\.libs\\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [3]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
df=pd.read_csv('https://github.com/AsmaMora/MADT8101/raw/main/5.Voice_of_customer/Got_to_grill_review.csv')

In [5]:
df.tail()

,ReviewID,Review
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...
9,10,เนื้อคือดีมาก


In [6]:
#Tokenize Words
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ','  ','\n','ค่ะ','คะ','ร้าน','นะคะ','ๆ','('')','(',')','(' ,')','(' , ')','(', ')','โอเค','"','"', 'กก' , 'Got' , 'To' , 'Grill']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize(str(sentence), engine='newmm')
    for word in words:
        if word not in screening_words:
            merged = merged + ',' + word
    return merged[1:]

In [7]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [8]:
df.tail()

,ReviewID,Review,Review_tokenized
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน,"นั่ง,อาหาร,อร่อย,ไม่ต้อง,รอ"
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ,"อาหาร,ญี่ปุ่น,ปิ้ง,ย่าง,ฮาลาล,กรุงเทพ"
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ,"เนื้อ,นุ่ม,พนักงานบริการ,นิดนึง"
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...,"บุฟเฟ่ต์,ฮาลาล,เนื้อ,ย่าง,พรีเมียม,สาย,เนื้อ,พ..."
9,10,เนื้อคือดีมาก,"เนื้อ,ดีมาก"


In [9]:
#Create Dictionary
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [10]:
print(dictionary.token2id.keys())

dict_keys(['Garden', 'Loft', 'บุฟเฟต์', 'พรีเมี่ยม', 'ราคา', 'อร่อย', 'ฮาลาล', 'แนว', 'ซี', 'ซูชิ', 'ถ้วม', 'บุ', 'ฟู้ด', 'ฟเฟ่ห์', 'ย่าง', 'เนื้อ', 'ไอศครีม', 'กะปิ', 'กิว', 'คุณภาพดี', 'บุฟเฟ่ต์', 'ม่อน', 'ย่าน', 'รามคำแหง', 'ล', 'วา', 'สาย', 'แซ', '399', 'คุ้ม', 'โปร', 'ซา', 'ซิ', 'ดีมาก', 'สด', 'เมนู', 'เลือก', 'นั่ง', 'รอ', 'อาหาร', 'ไม่ต้อง', 'กรุงเทพ', 'ญี่ปุ่น', 'ปิ้ง', 'นิดนึง', 'นุ่ม', 'พนักงานบริการ', 'to', 'พรีเมียม', 'พลาด'])


In [11]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [12]:
#Topic Modeling
num_topics = 4
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every =1 # Don't evaluate model perplexity, takes too much time

#make a index to word dictionary
temp = dictionary[0] #This is only to load the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                    alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, \
                                    passes=passes, eval_every=eval_every)

CPU times: total: 62.5 ms
Wall time: 113 ms


In [13]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.120872 -0.018557       1        1  43.346309
2     -0.092579 -0.060243       2        1  34.722205
1     -0.033600  0.096443       3        1  15.968236
3      0.005307 -0.017643       4        1   5.963250, topic_info=             Term      Freq     Total Category  logprob  loglift
15          เนื้อ  4.000000  4.000000  Default  30.0000  30.0000
39          อาหาร  1.000000  1.000000  Default  29.0000  29.0000
46  พนักงานบริการ  0.000000  0.000000  Default  28.0000  28.0000
44         นิดนึง  0.000000  0.000000  Default  27.0000  27.0000
45           นุ่ม  0.000000  0.000000  Default  26.0000  26.0000
..            ...       ...       ...      ...      ...      ...
14           ย่าง  0.062344  2.395971   Topic4  -4.1896  -0.8293
26            สาย  0.062344  1.879708   Topic4  -4.1896  -0.5867
20       บุฟเฟ่ต์  0.062344  1.879706   Topic4  -4.1896  -0.5867
8              ซี  0.062344  1.927119   Topic4  -4.1896  -0.6116
12           ฟู้ด  0.062344  1.927119   Topic4  -4.1896  -0.6116

[182 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
28        2  0.850375        399
0         2  0.850375     Garden
1         2  0.850374       Loft
47        1  0.817432         to
41        3  1.011690    กรุงเทพ
17        2  0.850374       กะปิ
18        2  0.850373        กิว
19        2  0.850374   คุณภาพดี
29        2  0.850375       คุ้ม
31        1  0.817426         ซา
32        1  0.817426         ซิ
8         1  1.037819         ซี
9         1  0.817426       ซูชิ
42        3  1.011690    ญี่ปุ่น
33        1  1.037820      ดีมาก
10        1  0.817426       ถ้วม
37        3  1.011689       นั่ง
11        1  0.817426         บุ
2         2  0.850375    บุฟเฟต์
20        1  0.531998   บุฟเฟ่ต์
20        2  0.531998   บุฟเฟ่ต์
43        3  1.011690       ปิ้ง
48        1  0.817431   พรีเมียม
3         2  0.850374  พรีเมี่ยม
49        1  0.817431       พลาด
12        1  1.037818       ฟู้ด
13        1  0.817426     ฟเฟ่ห์
21        2  0.850374       ม่อน
14        1  0.834735       ย่าง
14        3  0.417367       ย่าง
22        2  0.850374       ย่าน
38        3  1.011689         รอ
4         2  0.850375       ราคา
23        2  0.850374   รามคำแหง
24        2  0.850374          ล
25        2  0.850374         วา
34        1  0.817426         สด
26        1  0.531998        สาย
26        2  0.531998        สาย
5         2  0.434560      อร่อย
5         3  0.434560      อร่อย
39        3  0.686200      อาหาร
6         1  0.655237      ฮาลาล
6         2  0.327618      ฮาลาล
6         3  0.327618      ฮาลาล
15        1  0.809033      เนื้อ
15        2  0.202258      เนื้อ
35        1  0.817426       เมนู
36        1  0.817426      เลือก
27        2  0.850373         แซ
7         2  0.850375        แนว
30        2  0.850375        โปร
40        3  1.011690    ไม่ต้อง
16        1  0.817426    ไอศครีม, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

In [14]:
model.show_topic(1)

[('อาหาร', 0.09574447),
 ('ญี่ปุ่น', 0.05319135),
 ('ปิ้ง', 0.053191345),
 ('กรุงเทพ', 0.053191297),
 ('รอ', 0.053191226),
 ('ไม่ต้อง', 0.053191196),
 ('นั่ง', 0.053191137),
 ('อร่อย', 0.053190943),
 ('ย่าง', 0.053190406),
 ('ฮาลาล', 0.053190306)]

In [15]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [20]:
df.tail(10)

,ReviewID,Review,Review_tokenized,topics,score
0,1,"ร้าน ""Got To Grill"" พรีเมี่ยมฮาลาลบุฟเฟต์ที่แร...","พรีเมี่ยม,ฮาลาล,บุฟเฟต์,แนว,Loft,Garden,อร่อย,...",0,0.011188
1,2,บุฟเฟ่ห์เนื้อย่าง ซีฟู้ด ซูชิ ไอศครีมครบถ้วมแล...,"บุ,ฟเฟ่ห์,เนื้อ,ย่าง,ซี,ฟู้ด,ซูชิ,ไอศครีม,ถ้วม...",0,0.984131
2,3,บุฟเฟ่ต์สายเนื้อ คุณภาพดี วากิวอร่อยมากก แซลม่...,"บุฟเฟ่ต์,สาย,เนื้อ,คุณภาพดี,วา,กิว,อร่อย,แซ,ล,...",2,0.986224
3,4,โปร 399 ถือว่าคุ้มมาก,"โปร,399,คุ้ม",0,0.028371
4,5,เนื้อดีมาก มีเมนูให้เลือกเยอะมาก ซีฟู้ด ซาซิมิ สด,"เนื้อ,ดีมาก,เมนู,เลือก,ซี,ฟู้ด,ซา,ซิ,สด",0,0.982416
5,6,ร้านน่านั่ง อาหารอร่อย ไม่ต้องรอนาน,"นั่ง,อาหาร,อร่อย,ไม่ต้อง,รอ",0,0.017574
6,7,อาหารญี่ปุ่น ปิ้งย่างฮาลาล ที่สุดใน กรุงเทพ,"อาหาร,ญี่ปุ่น,ปิ้ง,ย่าง,ฮาลาล,กรุงเทพ",0,0.014765
7,8,เนื้อนุ่ม พนักงานบริการช้านิดนึง แต่ก็โอเคค่ะ,"เนื้อ,นุ่ม,พนักงานบริการ,นิดนึง",0,0.021706
8,9,บุฟเฟ่ต์ฮาลาล เนื้อย่างพรีเมียม สายเนื้อต้องไม...,"บุฟเฟ่ต์,ฮาลาล,เนื้อ,ย่าง,พรีเมียม,สาย,เนื้อ,พ...",0,0.982416
9,10,เนื้อคือดีมาก,"เนื้อ,ดีมาก",0,0.927835
